In [1]:
import cv2
import time
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist

ModuleNotFoundError: No module named 'cv2'

eye blinking

In [15]:
import cv2
import time
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist

# Initialize MediaPipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Eye aspect ratio calculation
def calculate_ear(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Eye landmark indices for the left and right eyes
LEFT_EYE_INDICES = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_INDICES = [362, 387, 385, 263, 373, 380]

# Parameters
EAR_THRESHOLD = 0.4  # Increased threshold to be more sensitive to blinks
CONSECUTIVE_FRAMES = 2  # Reduced number of frames for faster blink detection
BLINK_DURATION_THRESHOLD = 3  # Timer for the eye blink threshold in seconds

# Blink detection variables
blink_counter = 0
blink_total = 0
non_blinking_warning = False
last_blink_time = time.time()

# Start video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

            # Extract eye landmarks
            left_eye = np.array([(face_landmarks.landmark[i].x, face_landmarks.landmark[i].y) for i in LEFT_EYE_INDICES])
            right_eye = np.array([(face_landmarks.landmark[i].x, face_landmarks.landmark[i].y) for i in RIGHT_EYE_INDICES])

            # Calculate EAR for both eyes
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0

            # Detect blink
            if ear < EAR_THRESHOLD:
                blink_counter += 1
            else:
                if blink_counter >= CONSECUTIVE_FRAMES:
                    blink_total += 1
                    last_blink_time = time.time()  # Reset the blink timer
                    non_blinking_warning = False
                blink_counter = 0

            # Display EAR and blink count
            cv2.putText(frame, f"EAR: {ear:.2f}", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f"Blinks: {blink_total}", (30, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            # Determine if user is currently blinking
            if blink_counter >= CONSECUTIVE_FRAMES:
                cv2.putText(frame, "Blinking", (30, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Not Blinking", (30, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Check if the user hasn't blinked within the set threshold duration
            if time.time() - last_blink_time > BLINK_DURATION_THRESHOLD and not non_blinking_warning:
                cv2.putText(frame, "ALERT: Blink Your Eyes!", (30, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                non_blinking_warning = True

    # Display the image
    cv2.imshow('Face Liveliness Detection', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
